# **ETL Pipeline: Transferencia de Datos de Azure a Entorno Local.**
Este notebook es un **recurso de ayuda** diseñado para facilitar la **migración y sincronización de datos de manera sencilla** desde un servidor de Azure SQL hacia un entorno local SQL Server.

Este script es especialmente útil como base reutilizable para proyectos de integración de datos, pruebas, auditoría o cargas controladas entre entornos.

In [ ]:
import pyodbc
import pandas as pd
import numpy as np

In [ ]:
#  Conexión a Azure SQL.
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 18 for SQL Server}'

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

#  Conexión a SQL Server LOCAL.
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'  
LOCAL_DRIVER = '{ODBC Driver 18 for SQL Server}'

local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

#  Consulta SQL en Azure SQL.
SQL_QUERY = """
SELECT
"""

# Nombre de la tabla en SQL Server Local.
NEW_TABLE_NAME = "DATAEX.FACT_SALES"

try:
    #  Conectar a Azure SQL.
    print(f"Conectando a Azure SQL...")
    conn_azure = pyodbc.connect(azure_conn_str)
    
    # Ejecutar la consulta en Azure SQL.
    print(f"Ejecutando consulta en Azure SQL...")
    df = pd.read_sql(SQL_QUERY, conn_azure)

    if df.empty:
        print(f" La consulta no devolvió resultados. No se creará la tabla en SQL Server Local.")
    else:
        print(f"   - Datos extraídos: {df.shape[0]} filas")

        #  Convertir NaN en columnas numéricas a 0.
        df = df.fillna(0)

        #  Convertir valores numéricos problemáticos.
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = df[col].astype(np.float32)  # Reducir precisión

        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = df[col].astype(np.int32)  # Evitar valores fuera de rango

        #  Conectar a SQL Server Local.
        print(f"Conectando a SQL Server Local...")
        conn_local = pyodbc.connect(local_conn_str)

        with conn_local.cursor() as cursor:
            # Eliminar la tabla si ya existe.
            drop_table_sql = f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}"
            cursor.execute(drop_table_sql)
            conn_local.commit()
            print(f"   - Tabla eliminada si existía.")

            # Crear la tabla en SQL Server Local con tipos de datos ajustados.
            create_table_sql = f"""
            CREATE TABLE {NEW_TABLE_NAME} (
                {', '.join([
                    f'[{col}] FLOAT' if df[col].dtype == np.float32
                    else f'[{col}] INT' if df[col].dtype == np.int32
                    else f'[{col}] NVARCHAR(255)' for col in df.columns
                ])}
            );
            """
            cursor.execute(create_table_sql)
            conn_local.commit()
            print(f" Tabla {NEW_TABLE_NAME} creada correctamente en SQL Server Local.")

            # Insertar los datos en SQL Server Local
            placeholders = ', '.join(['?' for _ in df.columns])
            insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"
            cursor.fast_executemany = True
            cursor.executemany(insert_sql, df.values.tolist())
            conn_local.commit()
            print(f" {df.shape[0]} filas insertadas en {NEW_TABLE_NAME}.")

except Exception as e:
    print(f" Error: {e}")

finally:
    if 'conn_azure' in locals():
        conn_azure.close()
    if 'conn_local' in locals():
        conn_local.close()

print("\n ¡Proceso completado!")